# Binary Classification Dataset

### Importing Libraries and Setting Parameters

In [16]:
import os, warnings
import matplotlib.pyplot as plt
from matplotlib import gridspec
import random
from PIL import Image

import numpy as np
import pandas as pd
import tensorflow as tf
import keras
from tensorflow.keras.preprocessing import image_dataset_from_directory

# Reproducability
def set_seed(seed=31415):
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
set_seed(31415)

# Set Matplotlib defaults
plt.rc('figure', autolayout=True)
plt.rc('axes', labelweight='bold', labelsize='large',
       titleweight='bold', titlesize=18, titlepad=10)
plt.rc('image', cmap='magma')
warnings.filterwarnings("ignore") # to clean up output cells

### Listing image files and visualizing

In [15]:
def list_png_files(root_dir):
    png_files = []
    for subdir, dirs, files in os.walk(root_dir):
        for file in files:
            if file.lower().endswith('.png'):
                full_path = os.path.join(subdir, file)
                png_files.append(full_path)
    return png_files

train_dir = '/Users/miguelfa/Desktop/Bone-Fracture-Classifier/data/train'
val_dir = '/Users/miguelfa/Desktop/Bone-Fracture-Classifier/data/val'
test_dir = '/Users/miguelfa/Desktop/Bone-Fracture-Classifier/data/test'

train_png_files = list_png_files(train_dir)
val_png_files = list_png_files(val_dir)
test_png_files = list_png_files(test_dir)

In [19]:
def display_images(list_of_png):
    fig, axes = plt.subplots(nrows=len(list_of_png) // 5, ncols=5, figsize=(15, len(list_of_png) // 3))
    axes = axes.flatten()

    # Display each image
    for img_path, ax in zip(list_of_png, axes):
        image = Image.open(img_path)
        ax.imshow(image)
        ax.axis('off')
        ax.set_title(img_path.split('/')[-2])  # Title as the category

    plt.tight_layout()
    plt.show()

display_images(test_png_files)

### Parsing Images and Creating Datasets

In [34]:
# Decodes PNG, resizes Image and normalizes pixels from 0-1.
def parse_image(filename):
    parts = tf.strings.split(filename, os.sep)
    label = tf.cond(tf.strings.regex_full_match(parts[-2], 'fractured'),
                    lambda: 1,
                    lambda: 0)
    image = tf.io.read_file(filename)
    image = tf.image.decode_png(image, channels=3)
    image = tf.image.resize(image, [128, 128])
    image = image / 255.0
    return image, tf.cast(label, tf.int32)

# Creates dataset for each element in png list.
train_dataset = tf.data.Dataset.from_tensor_slices(train_png_files)
# Maps each dataset element to parse image function
train_dataset = train_dataset.map(parse_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
# buffered and batched
train_dataset = train_dataset.shuffle(buffer_size=1000).batch(64).prefetch(tf.data.experimental.AUTOTUNE)

val_dataset = tf.data.Dataset.from_tensor_slices(val_png_files)
val_dataset = val_dataset.map(parse_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
val_dataset = val_dataset.batch(64).prefetch(tf.data.experimental.AUTOTUNE)

test_dataset = tf.data.Dataset.from_tensor_slices(test_png_files)
test_dataset = test_dataset.map(parse_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
test_dataset = test_dataset.batch(64).prefetch(tf.data.experimental.AUTOTUNE)

In [23]:
train_dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

### Building Model

In [35]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, BatchNormalization, Dropout
from tensorflow.keras.models import Model
from tensorflow import keras
from tensorflow.keras import layers

image_classifier = keras.Sequential([
    
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=(128,128,3)),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.25),
    layers.Flatten(),
    layers.Dense(6, activation='relu'),
    Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

image_classifier.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_4 (Conv2D)               │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 124, 124, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 62, 62, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 62, 62, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 246016)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 6)              │     1,476,102 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 6)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │             7 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,495,501 (5.70 MB)

 Trainable params: 1,495,501 (5.70 MB)

 Non-trainable params: 0 (0.00 B)

In [36]:
# compiling the model
image_classifier.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['binary_accuracy'])

### Fitting and model training

In [38]:
history = image_classifier.fit(train_dataset,
                                            validation_data=val_dataset,
                                            epochs=30)

Epoch 1/30


2024-05-04 12:10:13.130626: W tensorflow/core/lib/png/png_io.cc:89] PNG warning: iCCP: known incorrect sRGB profile
2024-05-04 12:10:13.458899: W tensorflow/core/lib/png/png_io.cc:89] PNG warning: iCCP: known incorrect sRGB profile
2024-05-04 12:10:13.471079: W tensorflow/core/lib/png/png_io.cc:89] PNG warning: iCCP: profile 'ICC Profile': 0h: PCS illuminant is not D50
2024-05-04 12:10:13.802442: W tensorflow/core/lib/png/png_io.cc:89] PNG warning: iCCP: known incorrect sRGB profile


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 270ms/step - binary_accuracy: 0.6548 - loss: 0.4720

2024-05-04 12:10:16.247455: W tensorflow/core/lib/png/png_io.cc:89] PNG warning: iCCP: profile 'ICC Profile': 0h: PCS illuminant is not D50


2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 1s/step - binary_accuracy: 0.6543 - loss: 0.4739 - val_binary_accuracy: 0.7288 - val_loss: 0.4207
Epoch 2/30


...

2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 1s/step - binary_accuracy: 0.9295 - loss: 0.2257 - val_binary_accuracy: 1.0000 - val_loss: 0.0278
Epoch 30/30


2024-05-04 12:12:01.129277: W tensorflow/core/lib/png/png_io.cc:89] PNG warning: iCCP: known incorrect sRGB profile
2024-05-04 12:12:01.469597: W tensorflow/core/lib/png/png_io.cc:89] PNG warning: iCCP: known incorrect sRGB profile
2024-05-04 12:12:01.480757: W tensorflow/core/lib/png/png_io.cc:89] PNG warning: iCCP: profile 'ICC Profile': 0h: PCS illuminant is not D50
2024-05-04 12:12:01.790252: W tensorflow/core/lib/png/png_io.cc:89] PNG warning: iCCP: known incorrect sRGB profile


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 305ms/step - binary_accuracy: 0.9644 - loss: 0.1278

2024-05-04 12:12:04.016656: W tensorflow/core/lib/png/png_io.cc:89] PNG warning: iCCP: profile 'ICC Profile': 0h: PCS illuminant is not D50


2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 1s/step - binary_accuracy: 0.9629 - loss: 0.1285 - val_binary_accuracy: 0.9661 - val_loss: 0.0495


### Assessing Accuracy 

In [40]:
image_classifier_history = pd.DataFrame(history.history)

image_classifier_history.loc[:, ['loss', 'val_loss']].plot()
image_classifier_history.loc[:, ['binary_accuracy', 'val_binary_accuracy']].plot()

<Axes: >

### Make Predictions

In [ ]:
_, test_accuracy = image_classifier.evaluate(test_dataset)
print(f"Test Accuracy: {test_accuracy}")

In [48]:
predictions = image_classifier.predict(test_dataset)
predicted_classes = np.where(predictions > 0.5, 0, 1).flatten()

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 816ms/step


In [49]:
predicted_classes

array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [ ]:
def display_predictions(list_png_files, actual, prediction):
    fig, axes = plt.subplots(nrows=len(list_png_files) // 5, ncols=5, figsize=(15, len(list_png_files) // 3))
    axes = axes.flatten()

    for i, ax in enumerate(axes):
        ax.imshow(list_png_files[i])
        ax.axis('off')
        
        pred_label = 'Fractured' if predictions[i] == 1 else 'Not Fractured'
        actual_label = 'Fractured' if actual[i] == 1 else 'Not Fractured'
        title = f'Predicted: {pred_label}\nActual: {actual_label}'
        ax.set_title(title, fontsize=10)
    
    plt.tight_layout()
    plt.show()

display_predictions(predicted_classes)